In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [3]:
 # Loading training model

model = load_model("model.keras")
model

/Users/sabbha/Python-AI/Py_DS_ML_NLP_Bootcamp/.pydsmlenv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


<Sequential name=sequential, built=True>

In [4]:
# Loading encodrs and scalers

with open("gender_encoder.pkl", "rb") as file:
    gender_encoder = pickle.load(file)

gender_encoder

LabelEncoder()

In [5]:
with open("ohe_geo.pkl", "rb") as file:
    ohe_geo = pickle.load(file)

ohe_geo

OneHotEncoder(sparse_output=False)

In [7]:
with open("scaler.pkl", "rb") as file:
    scaler = pickle.load(file)

scaler

StandardScaler()

In [21]:
#  Input sample
input_data = {
"CreditScore": 600,
"Geography": "France",
"Gender": "Male",
"Age":40,
"Tenure" :3,          
"Balance":60000 ,
"NumOfProducts":2,    
"HasCrCard":1,
"IsActiveMember":1,
"EstimatedSalary":50000 
}

In [22]:
# Encode Geogtraphy
geo_encoded = ohe_geo.transform([[input_data['Geography']]])
geo_encoded

/Users/sabbha/Python-AI/Py_DS_ML_NLP_Bootcamp/.pydsmlenv/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


array([[1., 0., 0.]])

In [23]:
geo_encoded_df = pd.DataFrame(geo_encoded, columns=ohe_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [24]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [25]:
input_df["Gender"] = gender_encoder.transform(input_df["Gender"])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [26]:
input_df = pd.concat([input_df.drop('Geography', axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [27]:
# Scaling the data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [28]:
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


array([[0.02629262]], dtype=float32)

In [29]:
prediction[0][0]

0.026292618

In [31]:
if prediction[0][0] > 0.5:
    print("The customer is likely to exit")
else:
    print("The customer is not likely to exit")

The customer is not likely to exit
